# SQL to Python
This notebook is for people with a solid undertsanding of SQL, trying to make a switch to Python for data science.

It shows how common SQL commands can be written in Python, and also shows how to start moving beyond just what you can do in SQL.

In [1]:
# importing pandas, a "SQL-like" Python library with dataframe objects built in.
import pandas as pd

Dataset is from kaggle.
https://www.kaggle.com/carrie1/ecommerce-data. You don't have to download it though, as it is already in the repo folder.

In [2]:
# Using pandas "read_csv' function to load the data.
data = pd.read_csv('ecommerce_data.csv', index_col=0, parse_dates=True)

Our data is e-commerce data from an e-commerce shop based in the UK. Most of the customers the company sells to are wholesalers.

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850,United Kingdom


## Making a local, server-less SQL database
[SQLite](https://www.sqlite.org/whentouse.html) is an easy-to-use SQL database system, that lets an entire database live in just one file on your computer, no server needed.

Here, we will make our own SQLite database in the repo folder, and populate with the same e-commerce data we just read into a pandas dataframe.
This way, we will be able to compare SQL commands (called to an actual SQL database) with their Python equvilents on the same data.

In [4]:
import sqlite3, os, os.path

In [5]:
# Seeing where our SQL database will be created.
os.getcwd()

'/Users/rachelberryman/Documents/From_SQL_to_Python'

In [6]:
# Checking to see if a database with the same name already exists.
if os.path.isfile("E_commerce_data.db"):
    os.remove("E_commerce_data.db")

# here, we are creating our new database, called E_commerce_data.db, and establishing a connection to it.
# We name this connection "conn". We will need to specify that connection in every SQL command we run,
# so that the queries are run on the correct database.
conn = sqlite3.connect("E_commerce_data.db")

In [7]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format="%m/%d/%Y %H:%M")

In [8]:
# populating our new SQL database with the data from our dataframe.
data.to_sql('data', conn, if_exists='replace', index=False)

Taking an initial look at our data. 

We have ~40,000 records.

In [9]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


Of this, about 36,000 of them are from the UK.

In [10]:
data[data['Country']=='United Kingdom'].describe()

,Quantity,UnitPrice,CustomerID
count,361878.000000,361878.000000,361878.000000
mean,11.077029,3.256007,15547.871368
std,263.129266,70.654731,1594.402590
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,14194.000000
50%,4.000000,1.950000,15514.000000
75%,12.000000,3.750000,16931.000000
max,80995.000000,38970.000000,18287.000000


Describe will calculate summary statistics for all of the numeric columns.
It is also a good way to see if there are non-numeric columns that are being misclassified. 
"Customer ID", which is made up of numbers, was throught to be a numeric column when the CSV was read.

## Question 1: How many countries does are company sell to?

In [11]:
len(data['Country'].unique())

# or:

len(set(data['Country']))

37

In [12]:
pd.read_sql_query("""
SELECT COUNT(DISTINCT Country)
FROM data
""", conn)

,COUNT(DISTINCT Country)
0,37


## Question 2: What are our top 10 countries in terms of items sold?

In [13]:
data.groupby(['Country']).Quantity.sum().sort_values(ascending=False).head(10)

Country
United Kingdom    4008533
Netherlands        200128
EIRE               136329
Germany            117448
France             109848
Australia           83653
Sweden              35637
Switzerland         29778
Spain               26824
Japan               25218
Name: Quantity, dtype: int64

In [14]:
pd.read_sql_query("""
SELECT Country, SUM(Quantity)
FROM data
GROUP BY Country
ORDER BY SUM(Quantity) desc
LIMIT 10
""", conn)

,Country,SUM(Quantity)
0,United Kingdom,4008533
1,Netherlands,200128
2,EIRE,136329
3,Germany,117448
4,France,109848
5,Australia,83653
6,Sweden,35637
7,Switzerland,29778
8,Spain,26824
9,Japan,25218




## Question 3: How much money did we make on stickers in 2017?

In [15]:
stickers = data[data['Description'].str.contains("STICKER SHEET")==True].reset_index()
stickers['Revenue'] = stickers['Quantity'] * stickers['UnitPrice']
stickers['Revenue'].sum()

1139.0

In [16]:
pd.read_sql_query("""
SELECT SUM(UnitPrice * Quantity)
FROM data
WHERE Description LIKE "%STICKER SHEET%"
""", conn)

,SUM(UnitPrice * Quantity)
0,1139.0


## Question 4: "How much money did we make in 2011 in the UK?"

In [17]:
uk_2011 = data.loc[(data['Country']=='United Kingdom') & (data['InvoiceDate'].dt.year==2011)] 
sum(uk_2011['UnitPrice'] * uk_2011['Quantity'])

6284073.65400224

In [18]:
# SQLite doesn't have a year() function, so we have to get creative. 
# In the WHERE clause of our query, we have to format the date to just include its year component.
# You can read about date formats in SQLite here: https://www.tutorialspoint.com/sqlite/sqlite_date_time.html
pd.read_sql_query("""
SELECT SUM(UnitPrice * Quantity)
FROM data
WHERE Country = "United Kingdom" AND strftime('%Y', InvoiceDate) = '2011'
""", conn)

,SUM(UnitPrice * Quantity)
0,6.284074e+06


EXERCISE: what you can't do with SQL: 
- moving beyond explanatory queries to predictive analytics: simple ML model
- Visualisations with matplotlib

## What you can't do with SQL: more advanced analytics and visualizations

Starting: Building a Customer Segmentation

In [19]:
data['Revenue'] = data['UnitPrice'] * data['Quantity']

Step 1: group orders together by customer, aggregating the total number of orders they made, total quantity of all goods they've purchased, and total revenue gained.

In [20]:
def create_customer_table(df):
    # Groups purchasing data by customer ID.
    
    # create number_orders df
    customer_df = df.groupby(by='CustomerID', as_index=False)['InvoiceNo'].count()
    
    # create total metrics: sum of float values
    for indicator in ['Quantity', 'Revenue']:
        new_col = df.groupby(by='CustomerID', as_index=False)[indicator].sum()
        customer_df = customer_df.merge(new_col, how='inner', right_on='CustomerID', left_on='CustomerID')
    
    customer_df.columns = ['CustomerID', 'NumberOrders', 'TotalQuantity', 'TotalRevenue']
    
    return customer_df

In [21]:
customer_df = create_customer_table(data)
customer_df.tail()

,CustomerID,NumberOrders,TotalQuantity,TotalRevenue
4367,18280,10,45,180.60
4368,18281,7,54,80.82
4369,18282,13,98,176.60
4370,18283,756,1397,2094.88
4371,18287,70,1586,1837.28


Step 2: Calculating some additional metrics

In [22]:
def calculate_customer_loyalty_metris(customer_df, df):
    # Step 1: create df with groupby for both the min and max invoice dates.
    first_order = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].min()
    first_order.columns = ['CustomerID', 'FirstOrder']
    
    last_order = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max()
    last_order.columns = ['CustomerID', 'LastOrder']
    
    # Using a merge to combine the 2. This is just like a join in SQL.
    customer_metrics = first_order.merge(last_order, how='inner', right_on='CustomerID', left_on='CustomerID')
    
    # Adding column to use when calculating
    customer_metrics['Current'] = pd.to_datetime((df['InvoiceDate']).max())
    customer_metrics['CustomerTimeLength'] = 1 + (customer_metrics['Current']
                                                  - customer_metrics['FirstOrder']).astype('timedelta64[D]')

    customer_metrics['DaysSinceLastOrder'] = 1 + (customer_metrics['Current'] -
                                                 customer_metrics['LastOrder']).astype('timedelta64[D]')

    # dropping column we only used in calculating
    customer_metrics.drop(['Current', 'FirstOrder', 'LastOrder'], axis=1, inplace=True)
    
    # joining the table with the customer_df table
    customer_df = customer_df.merge(customer_metrics, how='inner', right_on='CustomerID', left_on='CustomerID')

    return customer_df

In [23]:
customer_df = calculate_customer_loyalty_metris(customer_df, data)

# using iloc to select a random few rows to look at.
customer_df.iloc[3:10]

,CustomerID,NumberOrders,TotalQuantity,TotalRevenue,CustomerTimeLength,DaysSinceLastOrder
3,12349,73,631,1757.55,19.0,19.0
4,12350,17,197,334.40,310.0,310.0
5,12352,95,470,1545.41,297.0,36.0
6,12353,4,20,89.00,204.0,204.0
7,12354,58,530,1079.40,232.0,232.0
8,12355,13,240,459.40,214.0,214.0
9,12356,59,1591,2811.43,326.0,23.0


Now, we have more metrics to look into.

We can look at the average of each of these metrics, and define categories with them.

In [24]:
customer_df.describe()

,CustomerID,NumberOrders,TotalQuantity,TotalRevenue,CustomerTimeLength,DaysSinceLastOrder
count,4372.000000,4372.000000,4372.000000,4372.000000,4372.000000,4372.000000
mean,15299.677722,93.053294,1122.344007,1898.459701,225.779048,92.047118
std,1722.390705,232.471608,4672.790720,8219.345141,118.412473,100.765435
min,12346.000000,1.000000,-303.000000,-4287.630000,1.000000,1.000000
25%,13812.750000,17.000000,153.000000,293.362500,115.000000,17.000000
50%,15300.500000,42.000000,365.000000,648.075000,253.000000,50.000000
75%,16778.250000,102.000000,962.250000,1611.725000,332.000000,143.000000
max,18287.000000,7983.000000,196719.000000,279489.020000,374.000000,374.000000


For most of the metrics, the mean far exceeds the median. This means that there are probably a few big customers (outliers) oredering a lot, and providing a lot more business than the average.

In [25]:
def check_for_outliers(row, metric, cutoff):
    # helper function for creating customer segments
    if row[metric] > cutoff:
        return 'Yes'
    else:
        return 'No'

def create_customer_segments(customer_df):
    for metric in list(customer_df.columns)[1:]:
        cutoff = customer_df[metric].mean()
        customer_df[metric + 'Outlier'] = customer_df.apply(lambda x: check_for_outliers(x, metric, cutoff), axis=1)
    if 
    return customer_df

In [26]:
create_customer_segments(customer_df)

,CustomerID,NumberOrders,TotalQuantity,TotalRevenue,CustomerTimeLength,DaysSinceLastOrder,NumberOrdersOutlier,TotalQuantityOutlier,TotalRevenueOutlier,CustomerTimeLengthOutlier,DaysSinceLastOrderOutlier
0,12346,2,0,0.000000e+00,326.0,326.0,No,No,No,Yes,Yes
1,12347,182,2458,4.310000e+03,367.0,2.0,Yes,Yes,Yes,Yes,No
2,12348,31,2341,1.797240e+03,358.0,75.0,No,Yes,No,Yes,No
3,12349,73,631,1.757550e+03,19.0,19.0,No,No,No,No,No
4,12350,17,197,3.344000e+02,310.0,310.0,No,No,No,Yes,Yes
5,12352,95,470,1.545410e+03,297.0,36.0,Yes,No,No,Yes,No
6,12353,4,20,8.900000e+01,204.0,204.0,No,No,No,No,Yes
7,12354,58,530,1.079400e+03,232.0,232.0,No,No,No,Yes,Yes
8,12355,13,240,4.594000e+02,214.0,214.0,No,No,No,No,Yes
9,12356,59,1591,2.811430e+03,326.0,23.0,No,Yes,Yes,Yes,No
